
# Fermat–Weber problem in 2D with CVXPY

This notebook solves the *Fermat–Weber problem* (also called *Weighted Geometric Median*) in 2D. That is, given $m$ points $p_i \in \mathbb{R}^2$, and $m$ nonnegative weights $w_i \ge 0$, 

\begin{align*}
\text{minimize } & \sum_{i=1}^m w_i\,\|x - p_i\|_2\, \\
\text{subject to } & x \in \mathbb{R}^2. 
\end{align*}

This is a convex problem that can be modeled as a weighted sum of Euclidean norms (or via epigraph variables and convex constraints).

In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['svg']

## Helper functions
We provide utilities to:
- print solver results,
- plot the points, weights, and the solution. 

In [ ]:
def print_results(problem, x_opt):
    """Print solver status and optimal value; x_opt is an array of shape (2,)."""
    print(f"Status: {problem.status}")
    if problem.value is not None:
        print(f"Optimal objective value: {problem.value:.6g}")
    if x_opt is not None:
        print(f"x* = ({x_opt[0]:.6g}, {x_opt[1]:.6g})")


def plot_fw_instance(P, w, x_star):
    """Plot points and the optimal location x*. Marker size scales with weights."""
    P = np.asarray(P)
    w = np.asarray(w)
    fig, ax = plt.subplots(figsize=(6, 6))
    sizes = 80 * (0.2 + w / (w.max() if w.max() > 0 else 1.0))
    ax.scatter(P[:, 0], P[:, 1], s=sizes, c='tab:blue', label='Data points')
    if x_star is not None:
        for i in range(P.shape[0]):
            ax.plot([x_star[0], P[i, 0]], [x_star[1], P[i, 1]], color='0.6', lw=1, alpha=0.7)
        ax.scatter([x_star[0]], [x_star[1]], marker='*', s=200, c='tab:red', label='x* (solution)')
    for i, (x_i, y_i) in enumerate(P):
        ax.annotate(f"i={i}, w={w[i]:.2g}", (x_i, y_i), textcoords='offset points', xytext=(5, 5), fontsize=9)
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_title('Fermat–Weber problem: points and solution')
    ax.legend(loc='best')
    ax.set_aspect('equal', adjustable='box')
    ax.grid(True, ls=':', alpha=0.5)
    plt.show()


## CVXPY formulation

The following function `fermat_weber_2d()` constructs and solves the problem in *sum-of-norms* form. 
- input: a 2D point cloud $P$ of shape `(m,2)`;
- output: optimal location $x$, and the CVXPY `Problem` object. 

In [ ]:
def fermat_weber_2d(P, w): 
    x = cp.Variable(2, name='x')
    terms = []
    for i in range(P.shape[0]):
        # Each term is w[i] * ||x - P[i]||_2
        terms.append(w[i] * cp.norm(x - P[i], 2))
    objective = cp.Minimize(cp.sum(terms))
    prob = cp.Problem(objective)
    prob.solve()
    return (None if x.value is None else x.value.copy()), prob


## Example
We build a set of $m=6$ points in the unit square $[0,1]^2$ and a collection of weights, solve the problem, and visualize the solution.


In [ ]:

# Example data (reproducible)
np.random.seed(7)
P = np.array([[0.15, 0.20],
              [0.80, 0.30],
              [0.20, 0.90],
              [0.70, 0.80],
              [0.35, 0.55],
              [0.60, 0.10]])
w = np.array([1.0, 4.0, 1.5, 2.0, 3.0, 0.5])

x_star, prob = fermat_weber_2d(P, w)
print_results(prob, x_star)
plot_fw_instance(P, w, x_star)


## (Optional) Epigraph formulation
The following cell solves the same problem introducing auxiliary variables $t_i$ with constraints
$\|x-p_i\|_2 \le t_i$. The formulation is equivalent. 

In [ ]:
def fermat_weber_2d_epigraph(P, w): 
    m = P.shape[0]
    x = cp.Variable(2, name='x')
    t = cp.Variable(m, nonneg=True, name='t')
    constraints = [cp.norm(x - P[i], 2) <= t[i] for i in range(m)]
    objective = cp.Minimize(w @ t)
    prob = cp.Problem(objective, constraints)
    prob.solve()
    return (None if x.value is None else x.value.copy()), prob


In [ ]:
# Solve the instance
x_star_epi, prob_epi = fermat_weber_2d_epigraph(P, w)
print("Epigraph formulation:")
print_results(prob_epi, x_star_epi)


## (Optional) Sensitivity with respect to one weight
As a quick illustration, we scale the weight of point with index 1 and plot the **trajectory** of the solution $x^*(\alpha)$ as the factor $\alpha$ varies.


In [ ]:
idx = 1  # index of the point whose weight is scaled
alphas = np.linspace(4.0, 6.0, 18)
traj = []
for a in alphas:
    w_mod = w.copy()
    w_mod[idx] = a
    x_a, _ = fermat_weber_2d(P, w_mod)
    traj.append(x_a)
traj = np.array(traj)

fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(P[:, 0], P[:, 1], c='tab:blue', s=60, label='Data points')
ax.plot(traj[:, 0], traj[:, 1], '-o', ms=3, c='tab:orange', label='Trajectory of x*(alpha)')
ax.scatter([traj[0,0]], [traj[0,1]], marker='x', s=60, c='tab:green', label='Start')
ax.scatter([traj[-1,0]], [traj[-1,1]], marker='s', s=60, c='tab:purple', label='End')
ax.set_title("Sensitivity: path of the solution as w[1] varies")
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.legend(loc='best')
ax.set_aspect('equal', adjustable='box')
ax.grid(True, ls=':', alpha=0.5)
plt.show()


### Remarks
- The general convex solver works well on small/medium instances; for very large instances, specialized iterative methods like *Weiszfeld's method* can be considered. Here we stay within CVXPY. 
